In [5]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import Statevector


def equality_blackbox(qc, x, y, a, t):
    """
    Flip the global phase on |t⟩ (assumed to be |−⟩) iff the two
    n-bit registers x and y are identical.

    Parameters
    ----------
    qc : QuantumCircuit      circuit to which gates are added
    x  : list[Qubit]         first  n-qubit register
    y  : list[Qubit]         second n-qubit register
    a  : list[Qubit]         n-1 0-ancillas supplied by the caller
    t  : Qubit               phase-kickback qubit (already in |−⟩)
    """
    n = len(x)
    assert len(y) == n and len(a) == n - 1, "size mismatch"

    # compute on qbit level NOT (x[i] ⊕ y[i])
    for i in range(n):
        qc.cx(y[i], x[i])          #x[i] ⊕ y[i]
        qc.x(x[i])                 # NOT After

    #AND chain the result together, product chain
    toffoliGates = []
    qc.ccx(x[0], x[1], a[0])
    toffoliGates.append((x[0], x[1], a[0]))
    for i in range(1, n - 1):
        qc.ccx(a[i - 1], x[i + 1], a[i])
        toffoliGates.append((a[i - 1], x[i + 1], a[i]))
    final_eq = a[n - 2]

    # phase kick back
    qc.cx(final_eq, t)

    # reverse coompute toffoli chain
    for c1, c2, trg in reversed(toffoliGates):
        qc.ccx(c1, c2, trg)

    # reverse compute NOT (x[i] ⊕ y[i]) 
    for i in reversed(range(n)):
        qc.x(x[i])
        qc.cx(y[i], x[i])   

## manual test for n=2
def test_eq_blackbox_n2():
    bitpairs = ["00", "01", "10", "11"]

    for xb in bitpairs:
        for yb in bitpairs:
            #  register layout: x0 x1  y0 y1  a0  t
            q = QuantumRegister(6)
            qc = QuantumCircuit(q)

            # prepare |x y 0 1⟩ basis state
            if xb[0] == "1": qc.x(q[0])
            if xb[1] == "1": qc.x(q[1])
            if yb[0] == "1": qc.x(q[2])
            if yb[1] == "1": qc.x(q[3])
            qc.x(q[5]); qc.h(q[5])          # t = |−⟩

            psi_before = Statevector.from_instruction(qc)

            equality_blackbox(
                qc,
                x=[q[0], q[1]],
                y=[q[2], q[3]],
                a=[q[4]],
                t=q[5],
            )

            psi_after = Statevector.from_instruction(qc)
            phase = np.vdot(psi_before.data, psi_after.data).real
            expected = -1 if xb == yb else 1
            text = "equality" if xb == yb else "IN-equality" 
            text += " between x and y"
            flag = f"correct, {expected} sinals {text}" if np.isclose(phase, expected) else "NOPE"
            print(f"{flag}  x={xb}  y={yb}  phase={phase:.0f}")

            if xb == "10" and yb == "11":
                print("\n circuit for x=10, y=11:")
                print(qc.draw("text"))

test_eq_blackbox_n2()

correct, -1 sinals equality between x and y  x=00  y=00  phase=-1
correct, 1 sinals IN-equality between x and y  x=00  y=01  phase=1
correct, 1 sinals IN-equality between x and y  x=00  y=10  phase=1
correct, 1 sinals IN-equality between x and y  x=00  y=11  phase=1
correct, 1 sinals IN-equality between x and y  x=01  y=00  phase=1
correct, -1 sinals equality between x and y  x=01  y=01  phase=-1
correct, 1 sinals IN-equality between x and y  x=01  y=10  phase=1
correct, 1 sinals IN-equality between x and y  x=01  y=11  phase=1
correct, 1 sinals IN-equality between x and y  x=10  y=00  phase=1
correct, 1 sinals IN-equality between x and y  x=10  y=01  phase=1
correct, -1 sinals equality between x and y  x=10  y=10  phase=-1
correct, 1 sinals IN-equality between x and y  x=10  y=11  phase=1

 circuit for x=10, y=11:
       ┌───┐┌───┐┌───┐                    ┌───┐     ┌───┐
q43_0: ┤ X ├┤ X ├┤ X ├───────■─────────■──┤ X ├─────┤ X ├
       └───┘└─┬─┘├───┤┌───┐  │         │  ├───┤┌───┐└─┬─┘